In [22]:
import numpy as np
import pandas as pd
import pprint
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
import pickle

from scipy import stats
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error , r2_score
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression


from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MultiLabelBinarizer




In [23]:
df = pd.read_csv(r"C:\Users\ANTON\Documents\VS Code\Project-04\swiggy.csv")
pd.set_option('display.max_columns', None) 
df


,id,name,city,rating,rating_count,cost,cuisine,lic_no,link,address,menu
0,567335,AB FOODS POINT,Abohar,--,Too Few Ratings,₹ 200,"Beverages,Pizzas",22122652000138,https://www.swiggy.com/restaurants/ab-foods-po...,"AB FOODS POINT, NEAR RISHI NARANG DENTAL CLINI...",Menu/567335.json
1,531342,Janta Sweet House,Abohar,4.4,50+ ratings,₹ 200,"Sweets,Bakery",12117201000112,https://www.swiggy.com/restaurants/janta-sweet...,"Janta Sweet House, Bazar No.9, Circullar Road,...",Menu/531342.json
2,158203,theka coffee desi,Abohar,3.8,100+ ratings,₹ 100,Beverages,22121652000190,https://www.swiggy.com/restaurants/theka-coffe...,"theka coffee desi, sahtiya sadan road city",Menu/158203.json
3,187912,Singh Hut,Abohar,3.7,20+ ratings,₹ 250,"Fast Food,Indian",22119652000167,https://www.swiggy.com/restaurants/singh-hut-n...,"Singh Hut, CIRCULAR ROAD NEAR NEHRU PARK ABOHAR",Menu/187912.json
4,543530,GRILL MASTERS,Abohar,--,Too Few Ratings,₹ 250,"Italian-American,Fast Food",12122201000053,https://www.swiggy.com/restaurants/grill-maste...,"GRILL MASTERS, ADA Heights, Abohar - Hanumanga...",Menu/543530.json
...,...,...,...,...,...,...,...,...,...,...,...
148536,553122,The Food Delight,Yavatmal,--,Too Few Ratings,₹ 200,"Fast Food,Snacks",21522053000452,https://www.swiggy.com/restaurants/the-food-de...,"The Food Delight, 94MC+X35, New Singhania Naga...",Menu/553122.json
148537,562647,MAITRI FOODS & BEVERAGES,Yavatmal,--,Too Few Ratings,₹ 300,Pizzas,license,https://www.swiggy.com/restaurants/maitri-food...,"MAITRI FOODS & BEVERAGES, POLIC MITRYA SOCIETY...",Menu/562647.json
148538,559435,Cafe Bella Ciao,Yavatmal,--,Too Few Ratings,₹ 300,"Fast Food,Snacks",21522251000378,https://www.swiggy.com/restaurants/cafe-bella-...,"Cafe Bella Ciao, SHOP NO 2 NEMANI MARKET SBI S...",Menu/559435.json
148539,418989,GRILL ZILLA,Yavatmal,--,Too Few Ratings,₹ 250,Continental,21521251000241,https://www.swiggy.com/restaurants/grill-zilla...,"GRILL ZILLA, SHO NO 2/6, POSTEL GROUND CHOWPAT...",Menu/418989.json


In [24]:
#Id change to index
df = df.set_index("id", drop=True)

# rating

In [25]:

# Step 1: Replace junk values with NaN
df['rating'] = df['rating'].replace(['--', 'NEW', '', 'nan', 'None'], np.nan)

# Step 2: FORCE numeric conversion (this is the missing step)
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')

# Step 3: Fill NaN with median
df['rating'] = df['rating'].fillna(df['rating'].median())

# Step 4: Verify
print(df['rating'].isna().sum())   # 0
print(df['rating'].dtype)          # float64


0
float64


# rating_count

In [26]:
df['rating_count'].unique()


array(['Too Few Ratings', '50+ ratings', '100+ ratings', '20+ ratings',
       '500+ ratings', '1K+ ratings', '5K+ ratings', nan, '10K+ ratings'],
      dtype=object)

In [27]:

df['rating_count'] = (
    df['rating_count']
    .astype(str)
    .str.replace('+', '', regex=False)
    .str.replace(' ratings', '', regex=False)
    .replace('Too Few Ratings', np.nan)
    .replace({'K': '000'}, regex=True)
)

df['rating_count'] = pd.to_numeric(df['rating_count'], errors='coerce')
df['rating_count'].unique()


array([   nan,    50.,   100.,    20.,   500.,  1000.,  5000., 10000.])

In [28]:
median_val = df['rating_count'].median()
df['rating_count'] = df['rating_count'].fillna(median_val)
df['rating_count'].isna().sum()   # should be 0



np.int64(0)

# cost

In [29]:
df['cost'] = (
    df['cost']
    .astype(str)
    .str.replace(r'[^\d.]', '', regex=True)  # remove ₹, commas, text
)

df['cost'] = pd.to_numeric(df['cost'], errors='coerce').astype('Int64')
df['cost'].unique()


<IntegerArray>
[200, 100, 250, 150, 300, 700, 650, 400, 350,  60,
 ...
 345,  94, 501, 216, 625,  88, 101, 312, 448,  64]
Length: 364, dtype: Int64

# cost fill

In [30]:
median_val = df['cost'].median()
df['cost'] = df['cost'].fillna(median_val)
df['cost'].isna().sum() 


np.int64(0)

# cuisine

In [34]:
df['cuisine'] = df['cuisine'].fillna('')
df['cuisine'].isna().sum() 

np.int64(0)

In [35]:


# comma mattum allow, letters & spaces allow
df['cuisine'] = (
    df['cuisine']
    .str.replace(r'[^A-Za-z, ]', '', regex=True)  # remove unwanted special chars
    .str.replace(r'\s+', ' ', regex=True)         # extra spaces remove
    .str.strip()                                  # start/end spaces remove
)


In [36]:
df['cuisine'] = df['cuisine'].apply(
    lambda x: f"{x},{x}" if isinstance(x, str) and ',' not in x else x
)


# city

In [37]:
#df['city'] = df['city'].str.replace(r'[^a-zA-Z,\s]', '', regex=True)
df['city'] = (
    df['city']
    .astype(str)
    .str.replace(r'[^A-Za-z,\s]', '', regex=True)  # remove numbers & special chars
    .str.replace(r'\s+', ' ', regex=True)          # extra spaces remove
    .str.replace(r'\s*,\s*', ',', regex=True)      # comma spacing normalize
    .str.strip()
)


In [38]:
df['city'] = df['city'].astype(str).str.split(',').str[-1].str.strip()


# Cleaned CSV file

In [ ]:
#df.to_csv(r"C:\Users\ANTON\Desktop\Cleaned.csv",index=False)

# LabelEncoder

In [39]:

le = LabelEncoder()

df['city'] = le.fit_transform(df['city'])


In [17]:
#df['city'].isna().sum() 

In [40]:
df['cuisine_list'] = df['cuisine'].str.split(',')
mlb = MultiLabelBinarizer()

cuisine_encoded = mlb.fit_transform(df['cuisine_list'])
cuisine_df = pd.DataFrame(
    cuisine_encoded,
    columns=mlb.classes_,
    index=df.index
)
df = pd.concat([df, cuisine_df], axis=1)
df.drop(columns=['cuisine', 'cuisine_list'], inplace=True)

In [41]:
df

,name,city,rating,rating_count,cost,lic_no,link,address,menu,,Afghani,African,American,Andhra,Arabian,Asian,Assamese,Attractive Combos Available,Australian,Awadhi,BEVERAGE,Bakery,Bakery products,Bangladeshi,Barbecue,Bengali,Beverages,Bhutanese,Bihari,Biryani,Biryani Shivaji Military Hotel,Bowl Company,British,Burgers,Burmese,Cafe,Chaat,Chettinad,Chinese,Coastal,Code valid on bill over Rs,Combo,Continental,Default,Desserts,Discount offer from Garden Cafe Express Kankurgachi,European,Fast Food,Free Delivery Limited Stocks,French,German,Goan,Greek,Grill,Grocery products,Gujarati,Haleem,Healthy Food,Home Food,Hyderabadi,Ice Cream,Ice Cream Cakes,Indian,Indonesian,Italian,ItalianAmerican,Jain,Japanese,Juices,Kashmiri,Kebabs,Kerala,Keto,Khasi,Konkan,Korean,Lebanese,Lucknowi,MAX Combos per Order,Maharashtrian,Malaysian,Malwani,Mangalorean,Meat,Mediterranean,Mexican,Middle Eastern,Mongolian,Mughlai,Naga,Navratri Special,Nepalese,North Eastern,North Indian,Oriental,Oriya,Paan,PanAsian,Parsi,Pastas,Persian,Pizzas,Popular Brand Store,Portuguese,Punjabi,Rajasthani,Rayalaseema,SVANidhi Street Food Vendor,Salads,Seafood,Sindhi,Singaporean,Snacks,South American,South Indian,Spanish,Special Discount from Hotel Swagath,Sri Lankan,Steakhouse,Street Food,Sushi,Sweets,Tandoor,Tandoor,Telangana,TexMex,Thai,Thalis,Tibetan,To Pm,Tribal,Turkish,Use Code JUMBO to avail,Use code XPRESS to avail,Vietnamese,Waffle,indian
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
567335,AB FOODS POINT,0,4.0,50.0,200,22122652000138,https://www.swiggy.com/restaurants/ab-foods-po...,"AB FOODS POINT, NEAR RISHI NARANG DENTAL CLINI...",Menu/567335.json,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
531342,Janta Sweet House,0,4.4,50.0,200,12117201000112,https://www.swiggy.com/restaurants/janta-sweet...,"Janta Sweet House, Bazar No.9, Circullar Road,...",Menu/531342.json,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
158203,theka coffee desi,0,3.8,100.0,100,22121652000190,https://www.swiggy.com/restaurants/theka-coffe...,"theka coffee desi, sahtiya sadan road city",Menu/158203.json,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
187912,Singh Hut,0,3.7,20.0,250,22119652000167,https://www.swiggy.com/restaurants/singh-hut-n...,"Singh Hut, CIRCULAR ROAD NEAR NEHRU PARK ABOHAR",Menu/187912.json,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
543530,GRILL MASTERS,0,4.0,50.0,250,12122201000053,https://www.swiggy.com/restaurants/grill-maste...,"GRILL MASTERS, ADA Heights, Abohar - Hanumanga...",Menu/543530.json,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [19]:
# df[link_clean].nunique()
# df.isna().sum() 

In [ ]:
#df.to_csv(r"C:\Users\ANTON\Desktop\Encoded.csv",index=False)

# Columns

In [42]:
# 1) Blank / empty column names remove
df = df.loc[:, df.columns.notna()]
df = df.loc[:, df.columns.str.strip() != ""]

# 2) Unnamed columns remove
df = df.loc[:, ~df.columns.str.contains('^Unnamed', na=False)]

# 3) Time range columns remove
df = df.loc[:, ~df.columns.str.contains("To|PM|Pm", case=False, na=False)]

# 4) Specific columns remove
cols_to_remove = ["name", "lic_no", "link", "address", "menu"]
df.drop(columns=cols_to_remove, inplace=True, errors="ignore")

print("Columns dropped from existing df successfully")



Columns dropped from existing df successfully


In [ ]:
#df.to_csv(r"C:\Users\ANTON\Desktop\final.csv",index=False)

In [46]:
from sklearn.cluster import KMeans

# Choose number of clusters
k = 20

kmeans = KMeans(n_clusters=k, random_state=42)
df['cluster'] = kmeans.fit_predict(X)


In [47]:
from sklearn.metrics import silhouette_score

score = silhouette_score(X, df['cluster'])
print("Silhouette Score:", score)


Silhouette Score: 0.3397273732762949


In [1]:
import pandas as pd

cleaned = pd.read_csv(
    r"C:\Users\ANTON\Documents\VS Code\Project-04\Streamlitapp\Cleaned.csv"
)

encoded = pd.read_csv(
    r"C:\Users\ANTON\Documents\VS Code\Project-04\Streamlitapp\Encoded.csv"
)

cleaned.to_pickle(
    r"C:\Users\ANTON\Documents\VS Code\Project-04\Streamlitapp\Cleaned.pkl"
)

encoded.to_pickle(
    r"C:\Users\ANTON\Documents\VS Code\Project-04\Streamlitapp\Encoded.pkl"
)

print("Pickle files created successfully")


Pickle files created successfully
